# 1. Build Master Database

This notebook:
1. Reads raw CSV/Parquet from `data/raw/`
2. Cleans and filters
3. Engineers core feature flags
4. Writes out `data/processed/master.parquet`
5. Quick sanity checks


In [40]:
## 0. Import Libraries

import pandas as pd
from ydata_profiling import ProfileReport
from datetime import datetime

pd.set_option('display.max_columns', 100)  
pd.set_option('display.width', None)       
pd.set_option('display.max_rows', 1000)

In [42]:
pwd

'/Users/arvindpuri/Clays_LondonLAB_Project/code'

## 1. Extract
Read raw file into one DataFrame.


In [45]:
## 1. Read the CSV into a DataFrame with explicit dtypes
dtype_map = {
    'Context ID': str,
    'Booking ID': str,
    'Session ID': str,
    'Search Days Ahead': 'Int64',
    'Search Charge': 'float',
    'Search Charge Type': 'category',
    'Venue ID': str,
    'Venue Name': 'category',
    'Party Size': 'Int64',
    'Was Search Available': 'boolean',
    'Reservation Days Ahead': 'Int64',
    'Reservation Charge': 'float',
    'Reservation Charge Type': 'category',
    'Year': 'Int64',
    'Month': 'Int64',
    'Reservation Cost ($)': 'float',
    'Packages Cost ($)': 'float',
    'Add Ons Cost ($)': 'float',
    'Promo Code Discount ($)': 'float',
    'Total Cost ($)': 'float',
    'Deposit Amount': 'float',
}
df = pd.read_csv(
    '/Users/arvindpuri/Clays_LondonLAB_Project/data/raw/Clays_data.csv',
    dtype=dtype_map,
    encoding='latin1',
    low_memory=False
)

# 2. Safety copy
df.to_parquet('/Users/arvindpuri/Clays_LondonLAB_Project/data/raw/full_raw.parquet', index=False)

# 3. Standardize date/time columns
df['Search At'] = pd.to_datetime(df['Search At'], dayfirst=True, errors='coerce')
df['Search Date'] = pd.to_datetime(df['Search Date'], dayfirst=True, errors='coerce')
df['Reservation Date'] = pd.to_datetime(df['Reservation Date'], dayfirst=True, errors='coerce')
df['Reservation Datetime'] = pd.to_datetime(df['Reservation Datetime'], dayfirst=True, errors='coerce')

# 4. Drop rows without Context ID (must be a bug)
df = df.dropna(subset=['Context ID'])

# 5. Save cleaned DataFrame
df.to_parquet('/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/full_cleaned.parquet', index=False)

df.head()

/var/folders/qx/zlv3_dkd1v3863bflm3qj3180000gn/T/ipykernel_8743/4007434546.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Search Date'] = pd.to_datetime(df['Search Date'], dayfirst=True, errors='coerce')
/var/folders/qx/zlv3_dkd1v3863bflm3qj3180000gn/T/ipykernel_8743/4007434546.py:38: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Reservation Date'] = pd.to_datetime(df['Reservation Date'], dayfirst=True, errors='coerce')


,Context ID,Booking ID,Session ID,Search At,Search Date,Search Time,Search Time Iso,Search Days Ahead,Search Charge,Search Charge Type,Venue ID,Venue Name,Party Size,Source Path,Occasion ID,Was Search Available,Reservation Date,Reservation Time,Reservation Time Iso,Reservation Datetime,Reservation Days Ahead,Reservation Charge,Reservation Charge Type,Game Area,Was Package Required,Packages Available,Packages,Add Ons Available,Add Ons,Time Extension Available,Time Extension Selected,Personal Info Completed,Personal Info Completed At,Marketing Opt In,Reservation Notes,Promo Code,Promo Code Applied,Billing Info Completed,Customer ID,Booking Status,Reservation ID,Reservation Reference Code,Reservation Tags,Reservation Cost ($),Packages Cost ($),Add Ons Cost ($),Promo Code Discount ($),Total Cost ($),Deposit Amount,Year,Month
0,202405010126QNRLCH,202405010126QNRLCH,202405010126QNRLCH,2024-01-05 01:26:00,2024-05-03,7.740000e+13,21:30:00,2,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Clays Canary Wharf,3,/,NaN,False,2024-05-03,7.740000e+13,21:30:00,NaT,-101,10.0,person,peg,False,NaN,NaN,NaN,NaN,False,False,False,NaN,False,NaN,NaN,False,False,NaN,draft,NaN,NaN,NaN,30.0,0.0,0.0,0.0,30.0,30.0,2024,5
1,202405010512TSPQW5,202405010512TSPQW5,202405010512TSPQW5,2024-01-05 05:12:00,2024-05-18,6.480000e+13,18:00:00,17,14.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",10,/,NaN,False,2024-05-18,6.480000e+13,18:00:00,NaT,-86,14.0,person,peg,True,NaN,NaN,NaN,NaN,False,False,False,NaN,False,NaN,NaN,False,False,NaN,draft,NaN,NaN,NaN,140.0,0.0,0.0,0.0,140.0,140.0,2024,5
2,2024050105143Y5DJM,2024050105143Y5DJM,2024050105143Y5DJM,2024-01-05 05:14:00,2024-06-01,4.500000e+13,12:30:00,31,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Clays Canary Wharf,6,/,NaN,False,2024-06-01,4.500000e+13,12:30:00,NaT,-72,10.0,person,peg,True,NaN,BU,NaN,NaN,False,False,False,NaN,False,NaN,NaN,False,False,NaN,draft,NaN,NaN,NaN,60.0,180.0,0.0,0.0,240.0,240.0,2024,5
3,202405010514QFD3AA,202405010514QFD3AA,202405010514QFD3AA,2024-01-05 05:14:00,2024-05-18,5.940000e+13,16:30:00,17,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",10,/,NaN,False,2024-05-18,5.940000e+13,16:30:00,NaT,-86,10.0,person,peg,True,NaN,NaN,NaN,NaN,False,False,False,NaN,False,NaN,NaN,False,False,NaN,draft,NaN,NaN,NaN,100.0,0.0,0.0,0.0,100.0,100.0,2024,5
4,202405010531W0GWUE,202405010531W0GWUE,202405010531W0GWUE,2024-01-05 05:31:00,2024-05-01,5.040000e+13,14:00:00,0,0.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Clays Canary Wharf,2,/,NaN,False,2024-05-01,5.040000e+13,14:00:00,NaT,-103,0.0,person,peg,True,NaN,NaN,NaN,NaN,False,False,False,NaN,False,NaN,NaN,False,False,NaN,draft,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,2024,5


## 2. Raw Cleaning & Filtering
Business-rule filters:
- Drop `party_size > 20`
- Remove negative money & days ahead
- Cap `Search Days Ahead` at 99th percentile or 180 days
- Drop `Search At` before August 1st 2024 


In [47]:
## 2. Raw Cleaning & Filtering

df = pd.read_parquet("/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/full_cleaned.parquet")

# 2a. Drop red columns
red_columns = [
    'Reservation Time', 'Reservation Datetime', 'Reservation Charge Type', 'Game Area',
    'Marketing Opt In', 'Reservation Notes', 'Reservation ID', 'Reservation Tag',
    'Year', 'Month'
]
df = df.drop(columns=[col for col in red_columns if col in df.columns])
print(f"After dropping red columns: {df.shape[1]} columns remaining")

# 2b. Filter party size & days ahead
df = df[(df['Party Size']>0)&(df['Party Size']<=20)]
df = df[df['Search Days Ahead']>=0]
cap_days = min(180, df['Search Days Ahead'].quantile(0.99))
df['Search Days Ahead'] = df['Search Days Ahead'].clip(upper=cap_days)

# 2c. Drop `Search At` before August 1st 2024 
df["Search At"] = pd.to_datetime(df["Search At"], errors="coerce") # Ensure it's a datetime (safe step)
df = df[df["Search At"] >= "2024-08-01"] # Filter to keep only rows on or after August 1st, 2024

# 2d. Persist filtered data
df.to_parquet("/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/mid_step_filtered.parquet", index=False)
print(f"Rows after filtering: {len(df)}")

After dropping red columns: 42 columns remaining
Rows after filtering: 9350


In [48]:
df.head()

,Context ID,Booking ID,Session ID,Search At,Search Date,Search Time,Search Time Iso,Search Days Ahead,Search Charge,Search Charge Type,Venue ID,Venue Name,Party Size,Source Path,Occasion ID,Was Search Available,Reservation Date,Reservation Time Iso,Reservation Days Ahead,Reservation Charge,Was Package Required,Packages Available,Packages,Add Ons Available,Add Ons,Time Extension Available,Time Extension Selected,Personal Info Completed,Personal Info Completed At,Promo Code,Promo Code Applied,Billing Info Completed,Customer ID,Booking Status,Reservation Reference Code,Reservation Tags,Reservation Cost ($),Packages Cost ($),Add Ons Cost ($),Promo Code Discount ($),Total Cost ($),Deposit Amount
3210,202405080629SRNZIO,202405080629SRNZIO,202405080629SRNZIO,2024-08-05 06:29:00,2024-05-18,5.040000e+13,14:00:00,10,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",8,/,None,False,2024-05-18,14:00:00,-86,10.0,True,None,BU,None,None,False,False,False,None,None,False,True,202408121538JJCC0H,reservation_success,62KVSTKM,None,80.0,240.0,0.0,0.0,320.0,80.0
3211,202405080822G2JPIL,202405080822G2JPIL,202405080822G2JPIL,2024-08-05 08:22:00,2024-05-12,4.500000e+13,12:30:00,4,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",4,/,None,False,2024-05-12,12:30:00,-92,10.0,True,None,SF,None,None,False,False,False,None,None,False,True,202408121538LTGJWI,reservation_success,62L4TN46,None,40.0,100.0,0.0,0.0,140.0,140.0
3212,2024050809387LJA56,2024050809387LJA56,2024050809387LJA56,2024-08-05 09:38:00,2024-07-12,5.760000e+13,16:00:00,65,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Clays Canary Wharf,9,/,None,False,2024-07-12,16:00:00,-31,10.0,False,None,GO,None,None,False,False,False,None,None,False,True,2024081215383LIJS4,reservation_success,432VA732,None,90.0,360.0,0.0,0.0,450.0,112.5
3213,202405080955HWJ3LX,202405080955HWJ3LX,202405080955HWJ3LX,2024-08-05 09:55:00,2024-05-10,6.480000e+13,18:00:00,2,14.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",2,/,None,False,2024-05-10,18:00:00,-94,14.0,False,None,None,None,None,False,False,False,None,None,False,True,202408121538M594XT,reservation_success,62LAZVDP,None,28.0,0.0,0.0,0.0,28.0,28.0
3214,202405081004WXVWPT,202405081004WXVWPT,202405081004WXVWPT,2024-08-05 10:04:00,2024-05-26,5.400000e+13,15:00:00,18,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,"Clays, The City",8,/,None,False,2024-05-26,15:00:00,-78,10.0,False,None,None,None,None,False,False,False,None,None,False,True,202408121538XU3GHU,reservation_success,62LB3YKC,None,80.0,0.0,0.0,0.0,80.0,80.0


## 3. Slot-Level Identifiers & Time Features

In [50]:
## 3. Slot‐Level Identifiers & Time Features

# 3a. Load filtered data
df = pd.read_parquet("/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/mid_step_filtered.parquet")


## 3b. Convert boolean columns to 0/1
boolean_columns = [
    'Was Search Available',
    'Time Extension Available',
    'Time Extension Selected',
    'Personal Info Completed',
    'Personal Info Completed At',
    'Promo Code',
    'Promo Code Applied',
    'Billing Info Completed'
]

for col in boolean_columns:
    if col in df.columns:
        df[col] = df[col].apply(
            lambda x: 1 if pd.notnull(x) and str(x).lower() in ['yes', 'true', 't', 'y', '1'] else 0
        )
        print(f"Converted {col} to binary (0/1)")
        
        
# 3c. Canonical timestamps
df['search_datetime']      = pd.to_datetime(df['Search At'], errors='coerce')
df['reservation_datetime'] = pd.to_datetime(df['Reservation Date'], errors='coerce')


# 3d. Change column names "Search Date/Time" to "Search At Date/Time"
df['search_date_for'] = pd.to_datetime(df['Search Date'], dayfirst=True, errors='coerce').dt.date
df['search_hour_for'] = pd.to_datetime(df['Search Time Iso'],   dayfirst=True, errors='coerce').dt.hour

assert df['search_date_for'].notnull().all(), "Some booking-dates missing!"
print("✔ ‘at’ & ‘for’ columns created")


# 3e. Replace dollars with pounds
df.columns = [col.replace('($)', '(£)') for col in df.columns]
print("Changed $ to £ in column names")


# 3f. Create column "Process Time Completion" = "Personal Info Completed At" - "Search At Time"
if all(col in df.columns for col in ['Personal Info Completed At', 'Search At Time']):
    # Parse the timestamps
    completed_at = pd.to_datetime(df['Personal Info Completed At'], errors='coerce')
    
    # Get today's date as reference for times
    ref_date = datetime.now().date()
    
    # Convert times to datetime using reference date
    search_time_str = df['Search At Time'].astype(str)
    search_time = pd.to_datetime(ref_date.strftime('%Y-%m-%d') + ' ' + search_time_str, errors='coerce')
    
    # Calculate difference in minutes
    df['Process Time Completion'] = (completed_at - search_time).dt.total_seconds() / 60
    print("Created Process Time Completion column")
    

# 3g. Good Venue Names
venue_mapping = {
    "ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3BfVmVudWUYgICsreOh4wgM": "Canary Wharf",
    "ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3BfVmVudWUYgID4itGHgQgM": "The City",
    "ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3BfVmVudWUYgICexo6z2QsM": "Birmingham"
}
df["Venue Name"] = df["Venue ID"].map(venue_mapping)
    
    
# 3h. Derive day/hour/lead_time
df['search_date_at'] = df['search_datetime'].dt.date
df['search_hour_at'] = df['search_datetime'].dt.hour
df['lead_time']   = (df['reservation_datetime'] - df['search_datetime']).dt.days
df['day_of_week'] = df['search_datetime'].dt.dayofweek
df['day_of_week_for'] = df['Search Date'].dt.dayofweek
df['month_for']       = df['Search Date'].dt.month
df['is_weekend']  = df['day_of_week'].isin([5,6]).astype(int)
df['month']       = df['search_datetime'].dt.month

# assert (df['lead_time']<0).all(), "Negative lead_time found!"
print("✔ Time features added")


Converted Was Search Available to binary (0/1)
Converted Time Extension Available to binary (0/1)
Converted Time Extension Selected to binary (0/1)
Converted Personal Info Completed to binary (0/1)
Converted Personal Info Completed At to binary (0/1)
Converted Promo Code to binary (0/1)
Converted Promo Code Applied to binary (0/1)
Converted Billing Info Completed to binary (0/1)
✔ ‘at’ & ‘for’ columns created
Changed $ to £ in column names
✔ Time features added


/var/folders/qx/zlv3_dkd1v3863bflm3qj3180000gn/T/ipykernel_8743/178559046.py:34: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['search_hour_for'] = pd.to_datetime(df['Search Time Iso'],   dayfirst=True, errors='coerce').dt.hour


In [69]:
df.columns

Index(['Context ID', 'Booking ID', 'Session ID', 'Search At', 'Search Date',
       'Search Time', 'Search Time Iso', 'Search Days Ahead', 'price_shown',
       'Search Charge Type', 'Venue ID', 'Venue Name', 'Party Size',
       'Source Path', 'Occasion ID', 'avail_flag', 'Reservation Date',
       'Reservation Time Iso', 'Reservation Days Ahead', 'Reservation Charge',
       'Was Package Required', 'Packages Available', 'Packages',
       'Add Ons Available', 'Add Ons', 'Time Extension Available',
       'Time Extension Selected', 'Personal Info Completed',
       'Personal Info Completed At', 'Promo Code', 'Promo Code Applied',
       'Billing Info Completed', 'Customer ID', 'Booking Status',
       'Reservation Reference Code', 'Reservation Tags',
       'Reservation Cost (£)', 'Packages Cost (£)', 'Add Ons Cost (£)',
       'Promo Code Discount (£)', 'Total Cost (£)', 'Deposit Amount',
       'search_datetime', 'reservation_datetime', 'search_date_for',
       'search_hour_for', '

## Creating Db for K-means

In [77]:
booking_db = df.groupby(['Venue Name', 'month_for', 'day_of_week_for', 'search_hour_for']).size().reset_index(name='booking_count')

# Check the structure
print("\nBooking database structure:")
print(booking_db.head())
print(f"\nShape: {booking_db.shape}")
print(f"Unique venues: {booking_db['Venue Name'].unique()}")


Booking database structure:
     Venue Name  month_for  day_of_week_for  search_hour_for  booking_count
0    Birmingham         12                0               18              1
1    Birmingham         12                2               21              1
2    Birmingham         12                4               20              1
3    Birmingham         12                5               13              1
4  Canary Wharf          1                5               13              1

Shape: (806, 5)
Unique venues: ['Birmingham' 'Canary Wharf' 'The City']


In [81]:
booking_db.to_parquet('/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/booking_master_db.parquet',index=False)

In [57]:
venue_map = (
    df[["Venue ID", "Venue Name"]]
    .dropna()
    .drop_duplicates()
    .groupby("Venue ID")["Venue Name"]
    .first()
)

## 4. Booking Indicator & Rates

In [60]:
## 4. Booking Indicator & Rates
df['was_booked'] = df['Booking ID'].notnull().astype(int)

slot_agg = (
    df.groupby(['Venue Name', 'search_date_for', 'search_hour_for'], as_index=False)
      .agg(
          n_searches   = ('was_booked', 'size'),
          n_bookings   = ('was_booked', 'sum'),
          lead_time    = ('lead_time',  'median'),   # or 'mean'
          day_of_week  = ('day_of_week','first'),    # same within a group
          is_weekend   = ('is_weekend','first')
      )
)
slot_agg['booking_rate'] = slot_agg['n_bookings'] / slot_agg['n_searches']


print(f"✔ Aggregated into {len(slot_agg)} slots")

✔ Aggregated into 1957 slots


## 5. Price & Availability Features

In [63]:
## 5. Price & Availability Features

# rename raw columns
df = df.rename(columns={
  'Search Charge':'price_shown',
  'Was Search Available':'avail_flag'
})

price_avail = (
  df.groupby(['Venue Name','search_date_for','search_hour_for'], as_index=False)
    .agg(
      avg_price=('price_shown','mean'),
      pct_avail=('avail_flag','mean')
    )
)

print("✔ Price & availability computed")


✔ Price & availability computed


## 6. Capacity & Price Bounds

In [66]:
## 6. Capacity & Price Bounds

# 6a. Manual capacity lookup per venue
venue_caps = {
    "City": 12,           # 12 pegs
    "Canary Wharf": 8,    #  8 pegs
    "Birmingham": 4       #  4 pegs; arcade seats can be handled separately later
}

# expand to every hour (assuming same capacity all day)
venue_cap = pd.DataFrame([
    {"Venue Name": vid, "search_hour_for": h, "capacity": cap}
    for vid, cap in venue_caps.items()
    for h in range(24)
])

# 6b. Fallback price bounds from historical data
# here we just take the global min/max shown price per venue
price_bounds = (
    df
    .groupby("Venue Name")["price_shown"]
    .agg(min_price="min", max_price="max")
    .reset_index()
)

# 6c. Merge into slot_agg
slot_level = (
    slot_agg
      .merge(price_avail,   on=["Venue Name","search_date_for","search_hour_for"], how="left")
      .merge(venue_cap,     on=["Venue Name","search_hour_for"],             how="left")
      .merge(price_bounds,  on="Venue Name",                             how="left")
)

# Sanity checks
# assert slot_level["capacity"].notnull().all(), "Some capacities missing!"
print("✔ Capacity & fallback price bounds merged")


✔ Capacity & fallback price bounds merged


## 7. Holiday & Event Flags

Maybe do it later when Tom replies to the email

In [31]:
# # 7. Holiday & Special-Event Flags
# # We can pull from a calendar API or maintain a static CSV

# holidays = pd.read_csv('data/static/holidays.csv')  # columns: date, is_holiday
# events   = pd.read_csv('data/static/events.csv')    # columns: date, event_flag

# slot_level = (
#     slot_level
#     .merge(holidays, left_on='search_date', right_on='date', how='left')
#     .merge(events,   left_on='search_date', right_on='date', how='left')
# )
# slot_level['is_holiday']  = slot_level['is_holiday'].fillna(0).astype(int)
# slot_level['event_flag']  = slot_level['event_flag'].fillna(0).astype(int)
# slot_level = slot_level.drop(columns=['date_x','date_y'])
# print("✔ Holiday & event flags added.")


## 8. Write out Master & Slot-Level Artifacts

In [34]:
# 8. Write out artifacts for downstream
# 8a. Master = every raw search + all new columns
df.to_parquet('/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/master.parquet',             index=False)

# 8b. Slot-level summary for modeling
slot_level.to_parquet('../data/processed/slot_level.parquet', index=False)
slot_level.to_csv('../data/processed/slot_level.csv',         index=False)

print("✅ master.parquet and slot_levels written!")


✅ master.parquet and slot_levels written!


## 9. Quick QA
Ensure everything looks good.


In [36]:
# 9. Load
df2 = pd.read_parquet('/Users/arvindpuri/Clays_LondonLAB_Project/data/processed/master.parquet')
print(df2.info())
df2.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9350 entries, 0 to 9349
Data columns (total 53 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Context ID                  9350 non-null   object        
 1   Booking ID                  9333 non-null   object        
 2   Session ID                  9350 non-null   object        
 3   Search At                   9350 non-null   datetime64[ns]
 4   Search Date                 9350 non-null   datetime64[ns]
 5   Search Time                 9350 non-null   float64       
 6   Search Time Iso             9350 non-null   object        
 7   Search Days Ahead           9350 non-null   Int64         
 8   price_shown                 9350 non-null   float64       
 9   Search Charge Type          9350 non-null   category      
 10  Venue ID                    9350 non-null   object        
 11  Venue Name                  9350 non-null   object      

,Context ID,Booking ID,Session ID,Search At,Search Date,Search Time,Search Time Iso,Search Days Ahead,price_shown,Search Charge Type,Venue ID,Venue Name,Party Size,Source Path,Occasion ID,avail_flag,Reservation Date,Reservation Time Iso,Reservation Days Ahead,Reservation Charge,Was Package Required,Packages Available,Packages,Add Ons Available,Add Ons,Time Extension Available,Time Extension Selected,Personal Info Completed,Personal Info Completed At,Promo Code,Promo Code Applied,Billing Info Completed,Customer ID,Booking Status,Reservation Reference Code,Reservation Tags,Reservation Cost (£),Packages Cost (£),Add Ons Cost (£),Promo Code Discount (£),Total Cost (£),Deposit Amount,search_datetime,reservation_datetime,search_date_for,search_hour_for,search_date_at,search_hour_at,lead_time,day_of_week,is_weekend,month,was_booked
0,202405080629SRNZIO,202405080629SRNZIO,202405080629SRNZIO,2024-08-05 06:29:00,2024-05-18,5.040000e+13,14:00:00,10,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,8,/,None,0,2024-05-18,14:00:00,-86,10.0,True,None,BU,None,None,0,0,0,0,0,0,1,202408121538JJCC0H,reservation_success,62KVSTKM,None,80.0,240.0,0.0,0.0,320.0,80.0,2024-08-05 06:29:00,2024-05-18,2024-05-18,14,2024-08-05,6,-80.0,0,0,8,1
1,202405080822G2JPIL,202405080822G2JPIL,202405080822G2JPIL,2024-08-05 08:22:00,2024-05-12,4.500000e+13,12:30:00,4,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,4,/,None,0,2024-05-12,12:30:00,-92,10.0,True,None,SF,None,None,0,0,0,0,0,0,1,202408121538LTGJWI,reservation_success,62L4TN46,None,40.0,100.0,0.0,0.0,140.0,140.0,2024-08-05 08:22:00,2024-05-12,2024-05-12,12,2024-08-05,8,-86.0,0,0,8,1
2,2024050809387LJA56,2024050809387LJA56,2024050809387LJA56,2024-08-05 09:38:00,2024-07-12,5.760000e+13,16:00:00,65,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Canary Wharf,9,/,None,0,2024-07-12,16:00:00,-31,10.0,False,None,GO,None,None,0,0,0,0,0,0,1,2024081215383LIJS4,reservation_success,432VA732,None,90.0,360.0,0.0,0.0,450.0,112.5,2024-08-05 09:38:00,2024-07-12,2024-07-12,16,2024-08-05,9,-25.0,0,0,8,1
3,202405080955HWJ3LX,202405080955HWJ3LX,202405080955HWJ3LX,2024-08-05 09:55:00,2024-05-10,6.480000e+13,18:00:00,2,14.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,2,/,None,0,2024-05-10,18:00:00,-94,14.0,False,None,None,None,None,0,0,0,0,0,0,1,202408121538M594XT,reservation_success,62LAZVDP,None,28.0,0.0,0.0,0.0,28.0,28.0,2024-08-05 09:55:00,2024-05-10,2024-05-10,18,2024-08-05,9,-88.0,0,0,8,1
4,202405081004WXVWPT,202405081004WXVWPT,202405081004WXVWPT,2024-08-05 10:04:00,2024-05-26,5.400000e+13,15:00:00,18,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,8,/,None,0,2024-05-26,15:00:00,-78,10.0,False,None,None,None,None,0,0,0,0,0,0,1,202408121538XU3GHU,reservation_success,62LB3YKC,None,80.0,0.0,0.0,0.0,80.0,80.0,2024-08-05 10:04:00,2024-05-26,2024-05-26,15,2024-08-05,10,-72.0,0,0,8,1
5,202405081025XOLHUB,202405081025XOLHUB,202405081025XOLHUB,2024-08-05 10:25:00,2024-05-11,7.830000e+13,21:45:00,3,10.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,6,/,None,0,2024-05-11,21:45:00,-93,10.0,False,None,None,None,None,0,0,0,0,0,0,1,202408121538KZUQDO,payment_inititalised,None,None,60.0,0.0,0.0,0.0,60.0,60.0,2024-08-05 10:25:00,2024-05-11,2024-05-11,21,2024-08-05,10,-87.0,0,0,8,1
6,202405081048R6IBPV,202405081048R6IBPV,202405081048R6IBPV,2024-08-05 10:48:00,2024-05-09,6.930000e+13,19:15:00,1,14.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,Canary Wharf,2,/,None,0,2024-05-09,19:15:00,-95,14.0,False,None,None,None,None,0,0,0,0,0,0,1,202408121538SVFPVK,reservation_success,4332634H,None,28.0,0.0,0.0,0.0,28.0,28.0,2024-08-05 10:48:00,2024-05-09,2024-05-09,19,2024-08-05,10,-89.0,0,0,8,1
7,202405081055GIIU0T,202405081055GIIU0T,202405081055GIIU0T,2024-08-05 10:55:00,2024-05-10,7.200000e+13,20:00:00,2,14.0,person,ahNzfnNldmVucm9vbXMtc2VjdXJlchwLEg9uaWdodGxvb3...,The City,4,/,None,0,2024-05-10,20:00:00,-94,14.0,False

## 10. Data Quality Report
Double check how the data is looking like before moving forward.


In [89]:
# # 10. Export HTML summary as data_quality.html

# def generate_data_quality_report(
#     input_path: str = "../data/processed/master.parquet",
#     output_path: str = "../outputs/data_quality.html"
# ):
#     # 1. Load your feature-augmented data
#     df = pd.read_parquet(input_path)
    
#     # 2. Create a profiling report
#     profile = ProfileReport(
#         df,
#         title="Clays Data Quality Report",
#         explorative=True,
#         minimal=True  # set True for a slimmer report
#     )
    
#     # 3. Export to HTML
#     profile.to_file(output_path)
#     print(f"✅ Data-quality report written to {output_path}")

# if __name__ == "__main__":
#     generate_data_quality_report()